# <a id='toc1_'></a>[JO 2024 project](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [JO 2024 project](#toc1_)    
  - [Prelude](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Fonctions](#toc1_3_)    
  - [Data collect](#toc1_4_)    
    - [Extract one country data](#toc1_4_1_)    
    - [Extract All countries data](#toc1_4_2_)    
    - [Extract all data](#toc1_4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prelude](#toc0_)

Work in progress . . .  

Summarize about the project :
- Data source : <a href="https://www.olympic.org/news">International Olympic Committee</a>
- Data extract from : <a href="http://olympanalyt.com/OlympAnalytics.php">olympanalyt.com - mail : sportsencyclo@gmail.com</a>

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [84]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## <a id='toc1_3_'></a>[Fonctions](#toc0_)

In [85]:
def GetHTML(url) :
    """
    Needs : url website
    Return : HTML text
    """
    response = requests.get(url)

    if response.status_code == 200 :
        print("Response OK, continue.")
    else :
        print("Access impossible.")
    
    return response.text

In [86]:
def Getdataframe(html) :

    """
    Needs : HTML text 
            with response = requests.get(url).text
    Return : Dataframe with athlete AND sport description : medal, country, athlete_name, games, sport, event
    Requirement : Must to apply on website like : adress[COUNTRY]adress
                i.e : http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=[COUNTRY]&param_games=ALL&param_sport=ALL
    """

    # Converting text (html) in soup (text parsed)
    soup = BeautifulSoup(html, "html5lib") #parse

    # Finding the correct table according to description
    table = soup.find("table", class_="simpletable", style="text-align:left;")

    # Retrieving the list (find_all) of all rows
    rows = table.find_all("tr")

    # Creating the table (future dataframe)
    athletes = []

    # For each row (= athlete)
    for row in rows:
        #describe_athlete = row.text.split() # Get the text
        cells = row.find_all("td") # Retrieving the list (find_all) of all cells in the row

        if len(cells) == 7: # If there are 7 cells then it's an athlete
            medal = cells[0].find('img')['title'][0]  # Medal title
            country = cells[1].find('img')['title']  # Athlete's country
            athlete_name = cells[2].text.strip()  # Athlete's name
            games = cells[3].find('img')['title']  # Olympic Games
            sport = cells[4].find('img')['title']  # Sport
            event = cells[6].text.strip()  # Event

            # Add the information to the list of athletes
            athletes.append({
                'medal': medal,
                'country': country,
                'athlete_name': athlete_name,
                'games': games,
                'sport': sport,
                'event': event
                })
            #print(cells)

        else: # Otherwise it's not an athlete, move to the next row
            next

    return pd.DataFrame(athletes)


## <a id='toc1_4_'></a>[Data collect](#toc0_)

### <a id='toc1_4_1_'></a>[Extract one country data](#toc0_)

In [87]:
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"

In [88]:
# Try with one country - FRA (France)
html = GetHTML(url)

Response OK, continue.


In [89]:
# Try to get the DF on one country - FRA (France)
df_medalists = Getdataframe(html)

df_medalists.sample(5)

,medal,country,athlete_name,games,sport,event
844,1,France,ABALO LucFABREGAS LudovicGERARD VincentKARABAT...,Japan,Handball,handball
403,1,France,Jean-Claude KILLY,France,Alpine Skiing,downhill
210,2,France,Eugène CONSTANTErnest BARBEROLLEGeorges LECOIN...,France,Rowing,four-oared shell with coxswain
462,2,France,Angelo PARISI,United States,Judo,+ 95kg (heavyweight)
698,2,France,Benjamin DARBELET,China,Judo,60 - 66kg (half-lightweight)


### <a id='toc1_4_2_'></a>[Extract All countries data](#toc0_)

Scrape countries codes to loop on each of them (website url modification)  
Link of countries codes : <a href="http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"> Countries list</a>

In [90]:
# Countries list
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"

html = GetHTML(url)

Response OK, continue.


In [91]:
# Parsing text in Beautifulsoup object
soup = BeautifulSoup(html, "html5lib")

In [92]:
# Select the right table
table = soup.find_all("table", class_="simpletable", style="text-align:center;")[1] # 2nd table
rows = table.find_all("tr")
countries = []

for row in rows[2:]: # ignore the column names
    data = row.text.split()

    if len(data) == 5 :
        countries.append(
            {
                'Code':data[0], 
                'Country':data[1], 
                'Continent':data[2], 
                'Firstparticipation':data[3], 
                'Lastparticipation':data[4]
            }
        )
    else :
        next

df_countries = pd.DataFrame(countries)

In [93]:
df_countries.sample(5)

,Code,Country,Continent,Firstparticipation,Lastparticipation
180,ZZX,Mixed,Team,1896,1904
0,AFG,Afghanistan,Asia,1936,2021
164,UGA,Uganda,Africa,1956,2021
128,PLE,Palestine,Asia,1996,2021
62,GRE,Greece,Europe,1896,2022


### <a id='toc1_4_3_'></a>[Extract all data](#toc0_)

In [94]:
code_countries = df_countries["Code"]

i = 0
for country in code_countries :
    url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=" + country + "&param_games=ALL&param_sport=ALL"
    html = GetHTML(url)
    df_temp = Getdataframe(html)
    if i == 0 :
        df_olympic = df_temp
        i += 1
    else : 
        df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)

    #Delete these codes after tries
    i += 1
    if i > 5 :
        break

Response OK, continue.
Response OK, continue.
Response OK, continue.
Response OK, continue.
Response OK, continue.


In [95]:
df_olympic.sample(5)

,medal,country,athlete_name,games,sport,event
9,1,Algeria,Нурия Мерах-Бенида,Australia,Athletics,1500m
10,2,Algeria,Али Саид-Сиф,Australia,Athletics,5000m
17,2,Algeria,MAKHLOUFI Taoufik,Brazil,Athletics,800m
13,3,Algeria,Мохамед Аллалу,Australia,Boxing,60 - 63.5kg (light-welterweight)
16,1,Algeria,Taoufik MAKHLOUFI,Great Britain,Athletics,1500m
